## 구글 트렌드를 다른 시계열 데이터와 결합 

<img src="data/google_trend.png">

- 검색어 인기도가 이 세상에 의미하는 것은 무엇일까? 구글 트렌드를 통해 구글 검색어의 인기도에 대해 접근이 가능하다.
- 이번 시간을 통해 알아볼 사항은 아래와 같다 
    - '비트코인' 검색량과 비트코인 가격의 관련성 
    - 테슬라와 같은 인기 주식 검색량과 해당 주식 가격의 관련성 
    - '실업 수당'에 대한 검색이 미국의 실제 실업률과 어떤 차이를 보이는지 확인

- 이번 시간에서 배울 내용은 아래와 같다.
    - 동일한 주기로 다시 샘플링하고 반환하여 비교 가능한 시계열데이터를 만드는 방법(예를 들어 일별 데이터에서 월별 데이터로)
    - 범위 설정, 레이블, 선 스타일, 마커, 색상, 차트 해상도를 이용하여 맷플롯립 차트의 스타일 미세 조정
    - 시계열의 계절성을 시각적으로 알아볼 수 있도록 그리드를 사용해보기
    - 누락된 NaN인 값의 개수를 찾고 데이터프레임에서 NaN 값을 어떻게 찾는지
    - 차트 시간축의 스타일 지정을 더 잘 하기 위해 로케이터로 작업하는 방법

### 1. 데이터 탐색 - 구글 검색 데이터 이해하기 

In [1]:
# 필요한 모듈 임포트 
import pandas as pd 
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

#### 1-1. 테슬라 트렌드 데이터 

In [16]:
# 데이터 불러오기 1 - 테슬라 트렌드 데이터
df_tesla = pd.read_csv('data/TESLA Search Trend vs Price.csv')
print(df_tesla.shape)
df_tesla.head()

(124, 3)


,MONTH,TSLA_WEB_SEARCH,TSLA_USD_CLOSE
0,2010-06-01,3,4.766
1,2010-07-01,3,3.988
2,2010-08-01,2,3.896
3,2010-09-01,2,4.082
4,2010-10-01,2,4.368


- 테슬라 데이터의 경우 124 행과 3 열로 구성 되어 있다. 
- 테슬라 데이터의 컬럼 정보는 아래와 같다 
    - **MONTH**: 월별 날짜 
    - **TSLA_WEB_SEARCH**: 테슬라 웹 서치 결과(인기도)
    - **TSLA_USD_CLOSE**: 테슬라 주가 종가기준 가격

In [17]:
# 테슬라 데이터의 통계 설명 추출 
df_tesla.describe()

,TSLA_WEB_SEARCH,TSLA_USD_CLOSE
count,124.000000,124.000000
mean,8.725806,50.962145
std,5.870332,65.908389
min,2.000000,3.896000
25%,3.750000,7.352500
50%,8.000000,44.653000
75%,12.000000,58.991999
max,31.000000,498.320007


In [18]:
# 테슬라 트렌드 검색량 최대, 최소값 추출
print(f'Largest value for Tesla in Web Search: {df_tesla.TSLA_WEB_SEARCH.max()}')
print(f'Smallest value for Tesla in Web Search: {df_tesla.TSLA_WEB_SEARCH.min()}')

Largest value for Tesla in Web Search: 31
Smallest value for Tesla in Web Search: 2


- 데이터프레임에서 ".describe()" 함수를 사용하면 많은 통계 설명을 볼 수 있다.
    - count: 컬럼별 총 데이터 수
    - mean /std: 컬럼별 데이터 평균 / 표준편차 
    - min / max: 컬럼별 데이터 최소값 / 최대값
    - 25% / 50% / 75%: 백분위수의 각 지점으로, 분포를 반영해 평균을 보완하는 목적으로 사용.
- 테슬라의 최대 검색량은 "31"건, 최소 검색량은 "2"건으로 확인된다. 

#### 1-2. 실업률 트렌드 데이터

In [19]:
# 데이터 불러오기 - 실업률 트렌드 데이터 
df_unemployment = pd.read_csv('data/UE Benefits Search vs UE Rate 2004-19.csv')
print(df_unemployment.shape)
df_unemployment.head()

(181, 3)


,MONTH,UE_BENEFITS_WEB_SEARCH,UNRATE
0,2004-01,34,5.7
1,2004-02,33,5.6
2,2004-03,25,5.8
3,2004-04,29,5.6
4,2004-05,23,5.6


In [14]:
# 실업률 트렌드 데이터 통계 설명
df_unemployment.describe()

,UE_BENEFITS_WEB_SEARCH,UNRATE
count,181.000000,181.000000
mean,35.110497,6.217680
std,20.484925,1.891859
min,14.000000,3.700000
25%,21.000000,4.700000
50%,26.000000,5.400000
75%,45.000000,7.800000
max,100.000000,10.000000


In [27]:
# 실업률 트렌드 검색량 최대, 최소값 추출
print(f'Largest value for UE Benefits in Web Search: {df_unemployment.UE_BENEFITS_WEB_SEARCH.max()}')
print(f'Smallest value for UE Benefits in Web Search: {df_unemployment.UE_BENEFITS_WEB_SEARCH.min()}')

Largest value for UE Benefits in Web Search: 100
Smallest value for UE Benefits in Web Search: 14


- 실업률 데이터의 경우 181 행과 3 열로 구성 되어 있다.
- 실업률 데이터의 컬럼 정보는 아래와 같다.
    - MONTH: 월별 날짜
    - UE_BENEFITS_WEB_SEARCH: 실업률 웹 서치 결과(인기도)
    - UNRATE: 실재 실업률 
- 실업률의 최대 검색량은 "100"건, 최소 검색량은 "14"건으로 확인된다.

#### 1-3 .비트코인 트렌드 데이터

In [23]:
# 데이터 불러오기 3 - 비트코인 가격 데이터
df_btc_price = pd.read_csv('data/Daily Bitcoin Price.csv')
print(df_btc_price.shape)
df_btc_price.head()

(2204, 3)


,DATE,CLOSE,VOLUME
0,2014-09-17,457.334015,21056800.0
1,2014-09-18,424.440002,34483200.0
2,2014-09-19,394.795990,37919700.0
3,2014-09-20,408.903992,36863600.0
4,2014-09-21,398.821014,26580100.0


In [24]:
# 데이터 불러오기 4 - 비트코인 트렌드 데이터 
df_btc_search = pd.read_csv('data/Bitcoin Search Trend.csv')
print(df_btc_search.shape)
df_btc_search.head()

(73, 2)


,MONTH,BTC_NEWS_SEARCH
0,2014-09,5
1,2014-10,4
2,2014-11,4
3,2014-12,4
4,2015-01,5


In [25]:
# 비트코인 가격 데이터 통계 설명
df_btc_price.describe()

,CLOSE,VOLUME
count,2203.000000,2.203000e+03
mean,4429.421245,8.043622e+09
std,4148.150071,1.176529e+10
min,178.102997,5.914570e+06
25%,433.629502,6.029915e+07
50%,3637.520020,2.018890e+09
75%,7997.372803,1.322478e+10
max,19497.400391,7.415677e+10


In [26]:
# 비트코인 트렌드 데이터 통계 설명
df_btc_search.describe()

,BTC_NEWS_SEARCH
count,73.000000
mean,15.013699
std,15.146959
min,3.000000
25%,5.000000
50%,14.000000
75%,18.000000
max,100.000000


In [29]:
# 비트코인 트렌드 검색량 최대, 최소값 추출
print(f'Largest value for BTC News Search in Web Search: {df_btc_search.BTC_NEWS_SEARCH.max()}')
print(f'Smallest value for BTC News Searc in Web Search: {df_btc_search.BTC_NEWS_SEARCH.min()}')

Largest value for BTC News Search in Web Search: 100
Smallest value for BTC News Searc in Web Search: 3


- 비트코인 데이터는 총 2개의 데이터가 존재한다. 일별 비트코인의 종가와 거래량을 알 수 있는 "Daily Bitcoin Price.csv"와 비트코인의 구글 트렌드에서 나온 "Bitcoin Search Trend.csv" 이다.
- 먼저 일별 비트코인 종가 및 거래량 데이터의 컬럼의 의미는 아래와 같다.
    - DATE: 일별 날짜
    - CLOSE: 일별 비트코인의 종가
    - VOLUME: 일별 비트코인 거래량
- 비트코인 구글 트렌드 데이트의 컬럼의 의미는 아래와 같다.
    - MONTH: 월별 날짜
    - BTC_NEWS_SEARCH: 비트코인 뉴스 검색량 
- 비트코인 뉴스의 최대 검색량은 "100"건, 최소 검색량은 "3"건으로 확인된다.

#### 1-4. 검색 숫자가 의미하는 것은 무엇일까? 
- 데이터프레임에서 구글의 검색 관심도 범위는 0에서 100 사이라는 점을 알 수 있다. 그런데 이것이 의미하는 것은 무엇일까? 구글이 검색 관심도 값을 정의하는 법은 아래와 같다. 
    - 검색 관심도 값은 주어진 지역 및 시간에 대한 차트상의 최대치에 대한 상대적 검색 관심도를 나타낸다. 
    - 100의 값은 용어의 관심도가 가질 수 있는 최고치이다. 
    - 50의 값은 중간 정도 인기가 있음을 의미힌다. 
    - 0 점은 해당 용어에 대한 충분한 데이터가 없음을 의미힌다. 
- 기본적으로 실제 검색량은 공개된 정보가 아니며, 구글은 조정된 수치만을 제공한다. 
- 각각의 데이터 수치는 상대적 인기도를 비교하기 위해 해당 검색 지역 및 시간 범위 별로 나눠진다. 
- 검색어 각각에 대해 구글은 각 지역과 기간별 총 검색량과 우리가 검색한 검색어 간의 관련성을 찾아준다. 그 다음 이 모든 측정값을 단일 인기도 측정값으로 결합해주고 주제 전체에 걸쳐 값을 조정한다. 
- 즉, 구글 트렌드는 해당 주제와 관련된 검색이 몇 건 되었는지 정확한 수를 알려주기보다 보기 좋게 가공한 결과를 제시한다. 
- 다음은 해당 csv 데이터를 생성하기 위해 사용했던 구글 트렌드 검색 매개변수 이다. 
    -  "Tesla", 전세계, 웹 검색
    - "Bitcoin", 전세계, 뉴스 검색
    - "Unemployment Benefits", 미국, 웹 검색
    
<img src="data/google_parameter.png">